# Freeze Location Tracking: 2 Mice



In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
from mfreeze.oop_interface import FreezeDetector, LoctionTracker
from mfreeze.utils import crop_set_same

# Folders

Here you specify where the videos to analyse are on your computer and where to save the results of the analysis. You should have all of the videos to analyse in a folder (directory).

- **file_extention**: The file extention of the video files. Foe example `.mp4` or `.avi`.
- **input_dir**: The folders where all of the conditioning videos are located.
- **report_dir** : The folder where the results will be saved
- **first_mouse_role**: The name of the role played by the first mouse to be analysed.
- **second_mouse_role**: The name of the role played by the second mouse to be analysed.

In [5]:
# Change these!

file_extention = ".mp4"
input_dir = Path(r"C:\Users\rory\repos\ah_vids\raw_videos\OG")
report_dir = Path(r"C:\Users\rory\repos\ah_vids\output\OG\conditioning")
first_mouse_role = "obs"
second_mouse_role = "dem"

In [6]:
report_dir.mkdir(exist_ok=True)
done_vids = []
videos = list(input_dir.glob(f"*{file_extention}"))
video_index = 0

# 1. New Video Here

Running the below cell will move on to the next video

In [ ]:
try:
    current_video = str(videos[video_index])
    print(f"Current Video:\n\t{current_video}")
except IndexError:
    print("No more videos left to analyse!", "\n")
    print("Check the contents of `done_vids` to make "
          "sure all of the videos to be analysed have been.")

# First Mouse

### Adjustable Parameters

Change these values to alter the performance of the freeze detection / location tracking.

**Do not assume that things are working well**.
Choose some initial values for these parameters, run the analysis and examine the videos that are produced in `report_dir`. if they are not good, *adjust these parameters*.

- **start_frame**: All frames before this will be cropped out. It defaults to the first frame. If there is a lot of movement at the start of the video (for example, from placing the camera into place), it is a good idea to crop it out.
- **mouse1_freeze_threshold**: Freeze Threshold for the first mouse. If too many frames are being classified as freezes, *lower* this value.
- **mouse2_freeze_threshold**: Freeze Threshold for the second mouse. If too many frames are being classified as freezes, *lower* this value.

In [3]:
# Change these!

start_frame = 0
mouse1_freeze_threshold = -0.7
mouse2_freeze_threshold = -0.7

In [ ]:
ROLE = first_mouse_role
detector = FreezeDetector(
    current_video, 
    save_video_dir=report_dir,
    freeze_threshold=mouse1_freeze_threshold, 
    start_frame=start_frame,
    med_filter_size=5
)

## Cropping

Cropping only the area around the first mouse. To crop:

1. Run the cell below. You should see an image of a frame and some buttons below it.
2. Click this button
<img src="crop_tool.png">
3. Double click frame
4. Draw a box around the area to include in the analysis

In [ ]:
i, c  = detector.interactive_crop(frame=start_frame)
i

## Run the analysis

In [2]:
detector.detect_motion()
detector.detect_freezes()
detector.save_video()

tracker = LoctionTracker(
    current_video, 
    start_frame=start_frame, 
    save_video_dir=report_dir,
    reference_num_frames=1000,
)
tracker = crop_set_same(detector, tracker)
tracker.track_location()
tracker.save_video()

dff = detector.generate_report()
dft = tracker.generate_report()
dfm1 = (
    pd.merge(dff, dft, on=["frame", "time", "video_name", "fps"])
    .rename(columns=({"was_freezing": f"was_freezing_{ROLE}",
                     "x": f"x_{ROLE}",
                     "y": f"y_{ROLE}",
                     "distance": f"distance_{ROLE}"}))
)

NameError: name 'detector' is not defined

# Second Mouse

All we need to do is **crop**. This time, crop the chamber of the second mouse.

In [ ]:
ROLE = second_mouse_role
detector = FreezeDetector(
    current_video, 
    save_video_dir=report_dir,
    freeze_threshold=mouse2_freeze_threshold, 
    start_frame=start_frame,
    med_filter_size=5
)
i, c  = detector.interactive_crop(frame=start_frame)
i

In [ ]:
detector.detect_motion()
detector.detect_freezes()
detector.save_video()

tracker = LoctionTracker(
    current_video, 
    start_frame=start_frame, 
    save_video_dir=report_dir,
    reference_num_frames=1000,
)
tracker = crop_set_same(detector, tracker)
tracker.track_location()
tracker.save_video()

dff = detector.generate_report()
dft = tracker.generate_report()
dfm2 = (
    pd.merge(dff, dft, on=["frame", "time", "video_name", "fps"])
    .rename(columns=({"was_freezing": f"was_freezing_{ROLE}",
                     "x": f"x_{ROLE}",
                     "y": f"y_{ROLE}",
                     "distance": f"distance_{ROLE}"}))
)

## Generate Combined Report and Save Results

In [ ]:
(
    pd.merge(dfm1, dfm2)
    .to_csv(report_dir / f"{Path(current_video).stem}.csv", index=False)
)

# Move onto the Next Video

To move onto the next video, run the cell below then go back to the cell below **1. New Video Here**.

In [ ]:
done_vids.append(vid)
print(len(done_vids))

video_index += 1